# Dead Reckoning, "by hand", in Python
This sheet intends to show the steps to go through to calculate Estimated Altitude (He) and Azimuth (Z) from scratch, to come up with a **Line of Positions**, without navigation tables.  
You might want to get some after that ;)  
> **_Note_**: To comply with the logarithm tables, we will use here **_decimal_** logathims.  
> natural ones would do to, but this is done for clarity, so you can compare the numbers when using your tables.


The equations to solve are:  
$$
He = arcsin((sin(L).sin(D)) + (cos(L).cos(D).cos(AHL)))
$$

$$
Z = arctan\left(\dfrac{sin(AHL)}{(cos(D).tan(D))-(sin(L).cos(AHL))}\right)
$$
The required precision implie the usage of logarithms.

In [78]:
import math

In [79]:
# Utility function, Decimal to Sexagesimal
def dec_to_sex(v: float) -> str:
    abs_val: float = abs(v)
    int_value: float = math.floor(abs_val)
    dec: float = abs_val - int_value
    i: int = int(int_value)
    dec *= 60.0
    return f"{i}\u00b0{dec:02.02f}'"

A quick test...

In [80]:
num: float = 37.0 + (46.0 / 60.0)
print(f"{num} becomes {dec_to_sex(num)}")


37.766666666666666 becomes 37°46.00'


We start with the following values:  

Estimated position:  
$L = N 37^o 46'$  
$G = W 122^o 37'$

From the almanacs:  
$D = S 6^o 10.84'$  
$AHG = 135^o 03.5'$  
$AHL = 12^o 26.5'$

First we look for the Estimated Altitude He.
$$
He = arcsin((sin(L).sin(D)) + (cos(L).cos(D).cos(AHL)))
$$
We need to find the logarithms of:  
$ sin(L) = sin(37^o46') $  
$ sin(D) = sin(-6^o 10.84')$ rounded to $sin(-6^o 11')$  
$ cos(L) = cos(37^o 46') $. 
$ cos(D) = cos(-6^o 10.84')$ rounded to $cos(-6^o 11')$  
$ cos(AHL) = cos(12^o 26.5') $ rounded to $cos(12^o 26')$  

In [81]:
lat: float = 37.0 + (46.0 / 60.0);
lng: float = -(122.0 + (37.0 / 60.0));
dec: float = -(6.0 + (11.0 / 60.0));
ahl: float = 12.0 + (26.0 / 60.0);

### He, Direct Calculation

In [82]:
direct_sin_he: float = (math.sin(math.radians(lat)) * math.sin(math.radians(dec))) + (math.cos(math.radians(lat)) * math.cos(math.radians(dec)) * math.cos(math.radians(ahl)))
he_direct: float = math.asin(direct_sin_he)
print(f"He: {dec_to_sex(math.degrees(he_direct))}, Bam!")

He: 44°32.92', Bam!


### He, With Logarithms

Notice that we use the absolute values of the angles (for the logaritms to be happy).

In [83]:
log_sin_lat: float = math.log10(math.sin(math.radians(abs(lat))))
log_sin_dec: float = math.log10(math.sin(math.radians(abs(dec))))
log_cos_lat: float = math.log10(math.cos(math.radians(abs(lat))))
log_cos_dec: float = math.log10(math.cos(math.radians(abs(dec))))
log_cos_ahl: float = math.log10(math.cos(math.radians(abs(ahl))))

In [84]:
print(f"log(sin(L)) = {log_sin_lat}")
print(f"log(sin(D)) = {log_sin_dec}")
print(f"log(cos(L)) = {log_cos_lat}")
print(f"log(cos(D)) = {log_cos_dec}")
print(f"log(cos(AHL)) = {log_cos_ahl}")

log(sin(L)) = -0.2129313065378506
log(sin(D)) = -0.9677433114478232
log(cos(L)) = -0.10209183331126717
log(cos(D)) = -0.0025339566758692165
log(cos(AHL)) = -0.01030676749454674


In [85]:
log_sin_lat_sin_dec: float = log_sin_lat + log_sin_dec;
print(f"log(sin(L).sin(D)) = ({log_sin_lat}) + ({log_sin_dec}) = {log_sin_lat_sin_dec}")

log(sin(L).sin(D)) = (-0.2129313065378506) + (-0.9677433114478232) = -1.1806746179856737


In [86]:
sin_l_sin_d: float = pow(10, log_sin_lat_sin_dec);  # exponential base 10
print(f"sin(L).sin(D) = {sin_l_sin_d}")

sin(L).sin(D) = 0.06596679464156176


In [87]:
log_cos_l_cos_d_cos_AHL: float = log_cos_lat + log_cos_dec + log_cos_ahl;
print(f"log(cos(L).cos(D).cos(AHL)) = ({log_cos_lat}) + ({log_cos_dec}) + ({log_cos_ahl}) = {log_cos_l_cos_d_cos_AHL}")

log(cos(L).cos(D).cos(AHL)) = (-0.10209183331126717) + (-0.0025339566758692165) + (-0.01030676749454674) = -0.11493255748168313


In [88]:
cos_l_cos_d_cos_AHL: float = pow(10, log_cos_l_cos_d_cos_AHL); # exponential base 10
print(f"cos(L).cos(D).cos(AHL) = {cos_l_cos_d_cos_AHL}")

cos(L).cos(D).cos(AHL) = 0.7674806638207804


In [89]:
# this sign evaluation may need some love...
sign: int = 1 if (abs(lat) == lat) else -1
sign *= 1 if (abs(dec) == dec) else -1
sign *= 1 if (abs(ahl) == ahl) else -1

the_sum: float = sin_l_sin_d + (sign * cos_l_cos_d_cos_AHL)  # SIGN !!!
print(f"sin(L).sin(D) + cos(L).cos(D).cos(AHL) = {the_sum}")

print(f"He: {dec_to_sex(math.degrees(math.asin(the_sum)))}")

sin(L).sin(D) + cos(L).cos(D).cos(AHL) = -0.7015138691792187
He: 44°32.92'


### Z, direct

In [90]:
tan_z: float = math.sin(math.radians(ahl)) / ((math.cos(math.radians(lat)) * math.tan(math.radians(dec))) - (math.sin(math.radians(lat)) * math.cos(math.radians(ahl))))
z_direct = abs(math.atan(tan_z));
print(f"Z: {dec_to_sex(math.degrees(z_direct))}")

Z: 17°28.74'


### Z, with logarithms

We need extra values

In [91]:
log_tan_dec: float = math.log10(math.tan(math.radians(abs(dec))))
print(f"log(tan(D)) = {log_tan_dec}")

log(tan(D)) = -0.9652093547719539


In [92]:
log_sin_ahl: float = math.log10(math.sin(math.radians(abs(ahl))))
print(f"log(sin(AHL)) = {log_sin_ahl}")

log(sin(AHL)) = -0.6669489188608413


In [93]:
log_cos_l_tan_d = log_cos_lat + log_tan_dec
print(f"log(cos(L).tan(D)) = {log_cos_l_tan_d}")

log(cos(L).tan(D)) = -1.067301188083221


In [94]:
log_sin_l_cos_AHL = log_sin_lat + log_cos_ahl;
print(f"log(sin(L).cos(AHL)) = ({log_sin_lat}) + ({log_cos_ahl}) = {log_sin_l_cos_AHL}")

log(sin(L).cos(AHL)) = (-0.2129313065378506) + (-0.01030676749454674) = -0.22323807403239734


In [95]:
cos_l_tan_d: float = pow(10, log_cos_l_tan_d);
sin_l_cos_AHL: float = pow(10, log_sin_l_cos_AHL);
denom: float = - cos_l_tan_d - sin_l_cos_AHL;  # -1 * cosLtanD, different signes.
print(f"Denominator: ({cos_l_tan_d}) - ({sin_l_cos_AHL}) = {denom}")

Denominator: (0.08564436859457944) - (0.5980836442675889) = -0.6837280128621683


In [96]:
tan_z = pow(10, log_sin_ahl) / denom;
print(f"Z: {dec_to_sex(math.degrees(math.atan(abs(tan_z))))}")

Z: 17°28.74'


And like before:

In [97]:
print(f"Z becomes {dec_to_sex(180 + math.degrees(math.atan(abs(tan_z))))}")

Z becomes 197°28.74'


---
This concludes the Python demo.